In [ ]:
from google.colab import drive
drive.mount('/ME')
predir='/ME/My Drive/'

import sys
datadirs=predir+'Colab Notebooks/LLFA/'
sys.path.insert(1, datadirs)

savepath = datadirs+'save/'
datapath = datadirs+'data/'

Mounted at /ME


In [ ]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


## Set-up

In [ ]:
class PARS:
    def __init__(self, device, datapath, savepath, dataset = 'Cifar10', architecture = 'CONV', 
                 num_channel = 64, process='E2E', update='BP', optimizer='SGD', loss='CE', 
                 nonlinear='hardtanh', lr=0.1, epochs=100, batchsize=500, 
                 repeat = 5, loadnet=None):
        self.dataset = dataset # 'Cifar10', 'Cifar100'
        self.architecture = architecture # 'LW', 'CONV'
        self.NUM_CHANNEL = num_channel # for 'LW'
        self.process = process # 'E2E', 'GLL', 'RLL'
        self.update = update # 'BP', 'FA', 'UF', 'US'
        self.OPT = optimizer # 'SGD', 'Adam', Only SGD with RLL
        self.loss = loss # 'CE', 'Hinge'
        self.nonlinear = nonlinear # 'hartanh','tanh'
        self.LR = lr
        self.epochs = epochs # Epochs per layer
        self.batch_size = batchsize
        self.repeat = repeat
        self.device = device
        self.datapath = datapath
        self.savepath = savepath
        self.loadnet = loadnet
    
    def __str__(self):
        res = ""
        for key, val in self.__dict__.items():
            if key != 'loadnet':
                res += "{}: {}\n".format(key, val)
            else:
                res += "{}: {}\n".format(key, val.keys() if val else val)
        return res

In [ ]:
from utils import *
from setup_net import setup_net
import os

def main(pars):
    print(pars)
    expdir = pars.savepath+pars.architecture+"/"+pars.loss+"/"
    if not os.path.exists(expdir):
        os.makedirs(expdir)
    EXP_NAME = 'epoch_{}_{}_{}_{}_LR_{}_Epochs_{}'.format(pars.process, pars.update, pars.OPT, pars.nonlinear, pars.LR, pars.epochs)

    if pars.loss == 'Hinge':
        pars.criterion = HingeLoss(pars.device)
    else:
        pars.criterion = torch.nn.CrossEntropyLoss()

    dtype = torch.float32
    data = get_data(pars.datapath, pars.dataset, num_train=45000)
    
    test_acc_all = []
    netDict = []
    for rep in range(pars.repeat):
        print("\nRep {}".format(rep+1))

        net, classifier = setup_net(pars)
        if pars.loadnet:
            net.load_state_dict(pars.loadnet)
        print(net)
        print(classifier)
      
        val_loss = []
        val_acc = []
        lw_test_acc = []
        
        if pars.process == 'RLL':
            train_model_rand(data, net, classifier, pars, val_loss, val_acc)
            lw_test_acc = check_accuracy_rand(data[4], data[5], net, classifier, pars)
            print('Rep: {}, te.acc = {}'.format(rep+1, [round(x,4) for x in lw_test_acc]))    
        elif pars.process == 'GLL':
            for i in range(pars.NUM_LAYER):
                print('LAYER:%d'%i)
                fix = net[:i]
                print(fix)
                model = nn.Sequential(
                    net[i],
                    classifier[i]
                )            
                print(model)
                 
                train_model(data, fix, model, pars, val_loss, val_acc)
                test_acc = check_accuracy(data[4], data[5], fix, model, pars) 
                print('Rep: %d, Layer: %d, te.acc = %.4f' % (rep+1, i, test_acc))
                lw_test_acc.append(test_acc)
                print()
        else: # 'E2E'
            train_model(data, net, classifier, pars, val_loss, val_acc)
            test_acc = check_accuracy(data[4], data[5], net, classifier, pars)
            print('Rep: %d, te.acc = %.4f' % (rep+1, test_acc))
            lw_test_acc.append(test_acc)
        
        np.save(expdir+'loss_rep_{}_'.format(rep+1) + EXP_NAME, val_loss)
        np.save(expdir+'val.acc_rep_{}_'.format(rep+1) + EXP_NAME, val_acc)
        np.save(expdir+'te.acc_rep_{}_'.format(rep+1) + EXP_NAME, lw_test_acc) 
                
        test_acc_all.append(lw_test_acc)
        netDict.append(net.state_dict())

    print('\nAll reps test.acc:')
    for acc in test_acc_all:
        print(acc)
    np.save(expdir+'te.acc.all_' + EXP_NAME, test_acc_all)

    return netDict

# CONV - GLL - BP - LR_0.05

In [ ]:
pars = PARS(device, datapath, savepath)
pars.process = 'GLL'
pars.LR = 0.05
print(pars)

architecture: CONV
process: GLL
update: BP
OPT: SGD
loss: CE
nonlinear: hardtanh
LR: 0.05
epochs: 100
batch_size: 500
repeat: 5
device: cuda:0
datapath: /ME/My Drive/Colab Notebooks/LLFA/data/
savepath: /ME/My Drive/Colab Notebooks/LLFA/save/
cuDNN: False



In [ ]:
main(pars)

architecture: CONV
process: GLL
update: BP
OPT: SGD
loss: CE
nonlinear: hardtanh
LR: 0.05
epochs: 100
batch_size: 500
repeat: 5
device: cuda:0
datapath: /ME/My Drive/Colab Notebooks/LLFA/data/
savepath: /ME/My Drive/Colab Notebooks/LLFA/save/
cuDNN: False

Files already downloaded and verified
Files already downloaded and verified

Rep 1
Sequential(
  (layer0): Sequential(
    (conv): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (activation): Hardtanh(min_val=-1.0, max_val=1.0)
    (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer1): Sequential(
    (conv): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (activation): Hardtanh(min_val=-1.0, max_val=1.0)
    (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (conv): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (activation): Hardtanh(min_val=-1.0, max_v

# CONV - RLL - BP - LR_0.05

In [ ]:
pars = PARS(device, datapath, savepath)
pars.process = 'RLL'
pars.LR = 0.05
print(pars)

architecture: CONV
process: RLL
update: BP
OPT: SGD
loss: CE
nonlinear: hardtanh
LR: 0.05
epochs: 100
batch_size: 500
repeat: 5
device: cuda:0
datapath: /ME/My Drive/Colab Notebooks/LLFA/data/
savepath: /ME/My Drive/Colab Notebooks/LLFA/save/
cuDNN: False



In [ ]:
main(pars)

architecture: CONV
process: RLL
update: BP
OPT: SGD
loss: CE
nonlinear: hardtanh
LR: 0.05
epochs: 100
batch_size: 500
repeat: 5
device: cuda:0
datapath: /ME/My Drive/Colab Notebooks/LLFA/data/
savepath: /ME/My Drive/Colab Notebooks/LLFA/save/
cuDNN: False

Files already downloaded and verified
Files already downloaded and verified

Rep 1
Sequential(
  (layer0): Sequential(
    (conv): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (activation): Hardtanh(min_val=-1.0, max_val=1.0)
    (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer1): Sequential(
    (conv): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (activation): Hardtanh(min_val=-1.0, max_val=1.0)
    (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (conv): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (activation): Hardtanh(min_val=-1.0, max_v